In [4]:
import pandas as pd 
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error as rmse

## Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".
Download the data for January and February 2023.
Read the data for January. 

How many columns are there?

In [5]:
df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df.shape[1]

19

## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [6]:
df['duration'] = ((df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds()/60)

df.duration.std()

np.float64(42.59435124195458)

## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [7]:
df_excl = df.loc[df.duration.between(1,60,inclusive='both')]

df_excl.shape[0] / df.shape[0]

0.9812202822125979

## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
* Fit a dictionary vectorizer
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [8]:
categorical = ['PULocationID', 'DOLocationID']
target = 'duration'

X = df_excl[categorical].astype(str).to_dict(orient='records')
y = df_excl[target].values

dv = DictVectorizer()
X_train = dv.fit_transform(X)
X_train.shape[1]

515

## Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [9]:
lr = LinearRegression()
lr.fit(X_train,y)
y_train_pred = lr.predict(X_train)
rmse(y, y_train_pred)

7.649261934850555

## Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [10]:
import src.pipeline as ppl

In [18]:
df_processed = ppl.transform_raw_data(df)
X_train_dict, y_train = ppl.get_matrices(df_processed)
X_train, dv = ppl.vectorize_matrix(X_train_dict)
predict_evaluate(X_train, y_train)

7.649261934850555

In [ ]:
def predict_evaluate(X, y, lr:Union[LinearRegression|None]=None) -> tuple:
    if lr is None:
        lr = LinearRegression()
        lr.fit(X, y)
        
    y_pred = lr.predict(X)
    
    return rmse(y, y_pred)

In [3]:
PATH = " https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"

df_val = pd.read_parquet(PATH)
df_val['duration'] = ((df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime).dt.total_seconds()/60)
df_val_excl = df_val.loc[df_val['duration'].between(1,60,inclusive='both')]
X_val_dict = df_val_excl[categorical].astype(str).to_dict(orient='records')
y_val = df_val_excl[target].values
X_val = dv.transform(X_val_dict)
y_val_pred = lr.predict(X_val)
rmse(y_val, y_val_pred)

NameError: name 'categorical' is not defined

In [22]:
def calculate_duration(df:pd.DataFrame) -> pd.DataFrame:
    df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds()/60
    return df 

In [28]:
def filter_outliers(df:pd.DataFrame) -> pd.DataFrame:
    return df.loc[df['duration'].between(1,60,inclusive='both')]



In [27]:
df_val.pipe(calculate_duration).pipe(filter_outliers)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.40,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00,1.683333
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.90,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25,32.083333
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.00,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00,13.300000
5,1,2023-02-01 00:52:40,2023-02-01 01:07:18,1.0,5.10,1.0,N,148,236,1,21.90,3.50,0.5,5.35,0.0,1.0,32.25,2.5,0.00,14.633333
6,1,2023-02-01 00:12:39,2023-02-01 00:40:36,1.0,8.90,1.0,N,137,244,1,41.50,3.50,0.5,3.50,0.0,1.0,50.00,2.5,0.00,27.950000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913950,2,2023-02-28 23:46:00,2023-03-01 00:05:00,NaN,4.65,NaN,None,249,140,0,20.22,0.00,0.5,4.84,0.0,1.0,29.06,NaN,NaN,19.000000
2913951,2,2023-02-28 23:26:02,2023-02-28 23:37:10,NaN,2.47,NaN,None,186,79,0,13.66,0.00,0.5,2.65,0.0,1.0,20.31,NaN,NaN,11.133333
2913952,2,2023-02-28 23:24:00,2023-02-28 23:38:00,NaN,3.49,NaN,None,158,143,0,17.64,0.00,0.5,0.00,0.0,1.0,21.64,NaN,NaN,14.000000
2913953,2,2023-02-28 23:03:00,2023-02-28 23:10:00,NaN,2.13,NaN,None,79,162,0,13.56,0.00,0.5,2.63,0.0,1.0,20.19,NaN,NaN,7.000000


In [1]:
from src.transform import *

In [2]:


predict(X_val, y_val, lr)

NameError: name 'X_val' is not defined

In [39]:

vectorize_matrix(X_val_dict, dv)[1]


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2 stored elements and shape (1, 515)>